# Generate phrase parsings for multiple functions

In [1]:
import pickle
import pandas as pd
import numpy as np
import sys
from datetime import datetime
import collections
import networkx as nx
from pathlib import Path

# import custom Construction builders and rules
from cx_analysis.search import SearchCX
from cx_analysis.cx import Construction
from cx_analysis.build import CXbuilder, CXbuilderTF
import cx_analysis.graph_nav as nav
from dataset import build_dataset

# add grammar to path
sys.path.append('../cxs')
from word_grammar import Words
from phrase_grammar import Subphrases
from phrase_classes import SinglePhrase

# import Text-fabric data
import tf_tools.formatting as form
from tf_tools.load import load_tf
from tf_tools.tokenizers import tokenize_surface
from tf_tools.formatting import book2sbl

repo = Path('/Users/cody/github/CambridgeSemiticsLab/time_collocations')
data_path = repo.joinpath('data/advb_article')

TF, API, A = load_tf('nhead mother', hoist=globals())

This is Text-Fabric 8.3.3
Api reference : https://annotation.github.io/text-fabric/cheatsheet.html

123 features found and 4 ignored
  0.00s loading features ...
   |     0.00s Dataset without structure sections in otext:no structure functions in the T-API
  7.14s All features loaded/computed - for details use loadLog()


In [2]:
# configure visualizers for Construction objects

cx_show = SearchCX(A)
pretty, prettyconds, showcx, search = (
    cx_show.pretty, cx_show.prettyconds, 
    cx_show.showcx, cx_show.search
)

In [3]:
test_functs = {'Time', 'Subj', 'Objc', 'Loca', 'Adju', 'Cmpl'}

In [4]:
# build testset, we only want to collect single-atomed 
# phrases with no daughter relations and only 1 head word

test_phrases = []

for phrase in F.otype.s('phrase'):
    function = F.function.v(phrase)
    n_atoms = len(L.d(phrase, 'phrase_atom'))
    heads = E.nhead.t(phrase)
    n_heads = len(heads)
    first_head = heads[0]
    daughters = E.mother.t(phrase)
    mothers = E.mother.f(phrase)
    n_mothers = len(mothers)
    n_daughters = len(daughters)
    
    if all([
        function in test_functs,
        n_atoms == 1,
        n_heads == 1,
        n_daughters == 0,
        n_mothers == 0,
        F.typ.v(phrase) not in {'InrP', 'NegP', 'InjP', 'CP', 'VP'},
    ]):
        test_phrases.append(phrase)
        
print(len(test_phrases), 'phrases selected...')

84656 phrases selected...


In [5]:
# collect words to begin analysis with

phrase_sample = test_phrases

phrase_words = [    
    word for phrase in phrase_sample
        for word in L.d(phrase, 'word')
]
len(phrase_words)

158588

# Word Analysis

In [6]:
words = Words(A, context='phrase') # word CX builder

# analyze all matches; return as dict
start = datetime.now()
print(f'Beginning word construction analysis...')
wordcxs = words.cxdict(phrase_words)
print(f'\t{datetime.now() - start} COMPLETE \t[ {len(wordcxs)} ] words loaded')

Beginning word construction analysis...
	0:01:20.670284 COMPLETE 	[ 158588 ] words loaded


# Subphrase Analysis

In [7]:
subphrases = Subphrases(wordcxs, A, context='phrase')

### Troubleshoot

In [8]:
# test_small = subphrases.prep(227481)
# showcx(test_small, conds=True, condenseType='sentence')

In [9]:
# A.search('''

# phrase function=Objc|Adju
#     word pdp=prep
#     <: word pdp=prep prs#n/a|absent
    
# ''')

In [10]:
# test = subphrases.analyzestretch(L.d(783909, 'word'), debug=True)

# for res in test:
#     showcx(res, conds=True, condenseType='phrase',)

### Run on all phrases

In [11]:
silent=True

phrase2cxs = collections.defaultdict(list)
nocxs = []

# time it
start = datetime.now()

print(f'{datetime.now()-start} beginning subphrase analysis...')

for i, phrase in enumerate(phrase_sample):
     
    # analyze all known relas
    elements = L.d(phrase, 'word')
    
    # analyze with debug exceptions
    try:
        cxs = subphrases.analyzestretch(elements)
    except:
        nocxs.append(phrase)
        
        if not silent:
            sys.stderr.write(f'error on {phrase}\t{T.text(phrase)}\n')
            
        continue
        
#         sys.stderr.write(f'\nFAIL...running with debug...\n')
#         pretty(phrase)
#         subphrases.analyzestretch(elements, debug=True)
#         raise Exception('...debug complete...')

    # save those phrases that have no matching constructions
    if not cxs:
        nocxs.append(phrase)
    else:
        phrase2cxs[phrase] = cxs
        
    # report status
    if i % 2000 == 0 and i:
        print(f'\t{datetime.now()-start}\tdone with iter {i}/{len(phrase_sample)}')
        
print(f'{datetime.now()-start}\tCOMPLETE')
print('-'*20)
print(f'{len(phrase2cxs)} phrases matched with Constructions...')
print(f'{len(nocxs)} phrases not yet matched with Constructions...')

0:00:00.000049 beginning subphrase analysis...
	0:00:24.176697	done with iter 2000/84656
	0:00:46.890205	done with iter 4000/84656
	0:01:08.990461	done with iter 6000/84656
	0:01:34.395921	done with iter 8000/84656
	0:02:01.029133	done with iter 10000/84656
	0:02:31.656060	done with iter 12000/84656
	0:02:57.617278	done with iter 14000/84656
	0:03:24.688126	done with iter 16000/84656
	0:03:49.923625	done with iter 18000/84656
	0:04:14.882664	done with iter 20000/84656
	0:04:43.231652	done with iter 22000/84656
	0:05:09.617743	done with iter 24000/84656
	0:05:33.152270	done with iter 26000/84656
	0:05:58.148513	done with iter 28000/84656
	0:06:21.452151	done with iter 30000/84656
	0:06:44.537676	done with iter 32000/84656
	0:07:11.195198	done with iter 34000/84656
	0:07:36.730512	done with iter 36000/84656
	0:08:00.340505	done with iter 38000/84656
	0:08:26.196669	done with iter 40000/84656
	0:08:50.289354	done with iter 42000/84656
	0:09:10.949740	done with iter 44000/84656
	0:09:31.47

# Classifier

In [12]:
from pathlib import Path
import pickle
from dataset import build_dataset

In [13]:
# compile acceptable head lexemes from single-phrased CXs
good_heads = set()
for ph, cx_data in phrase2cxs.items():
    if len(cx_data) == 1:
        cx = cx_data[0]
        head = list(cx.getsuccroles('head'))[-1]
        good_heads.add(F.lex.v(head))

# tag the time cxs with classifications
sp = SinglePhrase(phrase2cxs.values(), good_heads, A)  
sp.label_cxs()

In [14]:
with open(data_path.joinpath('function_cxs.pickle'), 'wb') as outfile:
    pickle.dump(phrase2cxs, outfile)

## Build CSV Dataset

In [15]:
with open(data_path.joinpath('function_cxs.pickle'), 'rb') as infile:
    phrase2cxs = pickle.load(infile)

In [16]:
constructions = [c for cs in phrase2cxs.values() 
                     for c in cs
                     if F.language.v(int(c)) == 'Hebrew']
dataset = build_dataset(constructions, A.api)

In [17]:
dataset.to_csv(data_path.joinpath('function_data.csv'))

In [18]:
dataset.shape

(84274, 40)

In [19]:
dataset.head()

,function,ref,book,ph_type,head,text,token,clause,sentence,classi,...,demon_str,demon_dist,ordinal,ord_str,cl_kind,verb,tense,verb_lex,book_sbl,lang
node,,,,,,,,,,,,,,,,,,,,,
651542,Time,Gen 1:1,Genesis,prep_ph,ראשׁית,בְּרֵאשִׁ֖ית,ב.ראשׁית,בְּרֵאשִׁ֖ית בָּרָ֣א אֱלֹהִ֑ים אֵ֥ת הַשָּׁמַ֖י...,בְּרֵאשִׁ֖ית בָּרָ֣א אֱלֹהִ֑ים אֵ֥ת הַשָּׁמַ֖י...,single.prep.bare.øanchor,...,NaN,NaN,False,NaN,VC,True,qtl,ברא,Gen,Hebrew
651544,Subj,Gen 1:1,Genesis,cont,אלהים,אֱלֹהִ֑ים,אלהים,בְּרֵאשִׁ֖ית בָּרָ֣א אֱלֹהִ֑ים אֵ֥ת הַשָּׁמַ֖י...,בְּרֵאשִׁ֖ית בָּרָ֣א אֱלֹהִ֑ים אֵ֥ת הַשָּׁמַ֖י...,single.øanchor,...,NaN,NaN,False,NaN,VC,True,qtl,ברא,Gen,Hebrew
651547,Subj,Gen 1:2,Genesis,defi_ph,ארץ,הָאָ֗רֶץ,ה.ארץ,וְהָאָ֗רֶץ הָיְתָ֥ה תֹ֨הוּ֙ וָבֹ֔הוּ,וְהָאָ֗רֶץ הָיְתָ֥ה תֹ֨הוּ֙ וָבֹ֔הוּ,single.definite,...,NaN,NaN,False,NaN,VC,True,qtl,היה,Gen,Hebrew
651551,Subj,Gen 1:2,Genesis,cont,חשׁך,חֹ֖שֶׁךְ,חשׁך,וְחֹ֖שֶׁךְ עַל־פְּנֵ֣י תְהֹ֑ום,וְחֹ֖שֶׁךְ עַל־פְּנֵ֣י תְהֹ֑ום,single.bare.øanchor,...,NaN,NaN,False,NaN,NC,False,NaN,NaN,Gen,Hebrew
651554,Subj,Gen 1:2,Genesis,geni_ph,רוח,ר֣וּחַ אֱלֹהִ֔ים,רוח.אלהים,וְר֣וּחַ אֱלֹהִ֔ים מְרַחֶ֖פֶת עַל־פְּנֵ֥י הַמּ...,וְר֣וּחַ אֱלֹהִ֔ים מְרַחֶ֖פֶת עַל־פְּנֵ֥י הַמּ...,single.genitive,...,NaN,NaN,False,NaN,VC,True,ptcp,רחף,Gen,Hebrew


In [20]:
dataset[dataset.cardinal & (dataset.function!='Time')]

,function,ref,book,ph_type,head,text,token,clause,sentence,classi,...,demon_str,demon_dist,ordinal,ord_str,cl_kind,verb,tense,verb_lex,book_sbl,lang
node,,,,,,,,,,,,,,,,,,,,,
651632,Cmpl,Gen 1:9,Genesis,prep_ph,מקום,אֶל־מָקֹ֣ום אֶחָ֔ד,אל.מקום.אחד,יִקָּו֨וּ הַמַּ֜יִם מִתַּ֤חַת הַשָּׁמַ֨יִם֙ אֶ...,יִקָּו֨וּ הַמַּ֜יִם מִתַּ֤חַת הַשָּׁמַ֨יִם֙ אֶ...,single.prep.øanchor.quantified.cardinal,...,NaN,NaN,False,NaN,VC,True,yqtl,קוה,Gen,Hebrew
652735,Objc,Gen 4:19,Genesis,numb_ph,אשׁה,שְׁתֵּ֣י נָשִׁ֑ים,שׁנים.אשׁה,וַיִּֽקַּֽח־לֹ֥ו לֶ֖מֶךְ שְׁתֵּ֣י נָשִׁ֑ים,וַיִּֽקַּֽח־לֹ֥ו לֶ֖מֶךְ שְׁתֵּ֣י נָשִׁ֑ים,single.øanchor.quantified.cardinal,...,NaN,NaN,False,NaN,VC,True,wyqtl,לקח,Gen,Hebrew
653961,Cmpl,Gen 9:22,Genesis,prep_ph,אח,לִשְׁנֵֽי־אֶחָ֖יו,ל.שׁנים.אח,וַיַּגֵּ֥ד לִשְׁנֵֽי־אֶחָ֖יו בַּחֽוּץ׃,וַיַּגֵּ֥ד לִשְׁנֵֽי־אֶחָ֖יו בַּחֽוּץ׃,single.prep.quantified.cardinal,...,NaN,NaN,False,NaN,VC,True,wyqtl,נגד,Gen,Hebrew
654159,Subj,Gen 10:25,Genesis,numb_ph,בן,שְׁנֵ֣י בָנִ֑ים,שׁנים.בן,וּלְעֵ֥בֶר יֻלַּ֖ד שְׁנֵ֣י בָנִ֑ים,וּלְעֵ֥בֶר יֻלַּ֖ד שְׁנֵ֣י בָנִ֑ים,single.øanchor.quantified.cardinal,...,NaN,NaN,False,NaN,VC,True,qtl,ילד,Gen,Hebrew
654266,Subj,Gen 11:6,Genesis,numb_ph,עם,עַ֤ם אֶחָד֙,עם.אחד,הֵ֣ן עַ֤ם אֶחָד֙,הֵ֣ן עַ֤ם אֶחָד֙,single.øanchor.quantified.cardinal,...,NaN,NaN,False,NaN,NC,False,NaN,NaN,Gen,Hebrew
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
903465,Loca,2 Chr 33:5,2_Chronicles,prep_ph,חצר,בִּשְׁתֵּ֖י חַצְרֹ֥ות בֵּית־יְהוָֽה׃,ב.שׁנים.חצר.בית.יהוה,וַיִּ֥בֶן מִזְבְּחֹ֖ות לְכָל־צְבָ֣א הַשָּׁמָ֑י...,וַיִּ֥בֶן מִזְבְּחֹ֖ות לְכָל־צְבָ֣א הַשָּׁמָ֑י...,single.prep.genitive.quantified.cardinal,...,NaN,NaN,False,NaN,VC,True,wyqtl,בנה,2 Chr,Hebrew
904238,Objc,2 Chr 35:7,2_Chronicles,numb_ph,בקר,בָקָ֖ר שְׁלֹ֣שֶׁת אֲלָפִ֑ים,בקר.שׁלשׁ.אלף,וּבָקָ֖ר שְׁלֹ֣שֶׁת אֲלָפִ֑ים,וַיָּ֣רֶם יֹאשִׁיָּ֣הוּ לִבְנֵ֪י הָעָ֟ם צֹ֞אן ...,single.øanchor.quantified.cardinal,...,NaN,NaN,False,NaN,WP,False,NaN,NaN,2 Chr,Hebrew
904250,Objc,2 Chr 35:8,2_Chronicles,numb_ph,בקר,בָקָ֖ר שְׁלֹ֥שׁ מֵאֹֽות׃,בקר.שׁלשׁ.מאה,חִלְקִיָּ֨ה וּזְכַרְיָ֜הוּ וִֽיחִיאֵ֗ל נְגִידֵ...,חִלְקִיָּ֨ה וּזְכַרְיָ֜הוּ וִֽיחִיאֵ֗ל נְגִידֵ...,single.component.øanchor.quantified.cardinal,...,NaN,NaN,False,NaN,VC,True,qtl,נתן,2 Chr,Hebrew


In [21]:
dataset.ph_type.value_counts()

prep_ph       42360
cont          16867
name           6834
geni_ph        5503
prps           3624
defi_ph        3622
numb_ph        2197
prde            717
adjv_ph         580
prin            551
advb            499
attrib_ph       333
qquant          248
card            220
prep             41
card_chain       41
appo_name        21
ordn              9
demon_ph          6
intj              1
Name: ph_type, dtype: int64